Süper—**Credit Card Fraud (ULB)** için SOM egzersizini “baştan sona” tek parça, Jupyter’da çalışır **tam kod** olarak veriyorum.
Aşağıdaki betik; veri yükleme → **kimlik kolonu atma** → **ölçekleme** → **SOM eğitimi** → **U-Matrix** → **SOM tabanlı iki skor** (QE ve BMU-fraud-yoğunluğu) → **eşik seçimi (F1-opt / Precision≥x)** → **K-Means karşılaştırma** adımlarını içerir.
Her adımın **nedenini** yorum satırlarında belirttim.

> Notlar:
>
> * Dosya adı **`creditcard.csv`** varsayılmıştır.
> * `MiniSom` yoksa: `pip install minisom`
> * Grafikler için `matplotlib` kullanılıyor.

---

```python
# ===========================================
# 0) Kurulum ve log/uyarı ayarları (temizlik)
# ===========================================
import os, warnings, logging
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
warnings.filterwarnings("ignore")
logging.getLogger().setLevel(logging.ERROR)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (roc_auc_score, average_precision_score,
                             precision_recall_curve, roc_curve,
                             classification_report)

from minisom import MiniSom
from sklearn.cluster import KMeans
```

```python
# ===========================================
# 1) Veri: Yükle, hedef/özellik ayır, kimlik kolonu at
# ===========================================
# NEDEN? 'CustomerID' gibi kimlik kolonları davranış bilgisi içermez, uzaklık tabanlı yöntemleri bozar.
#       Bu yüzden mutlaka çıkarıyoruz.

CSV_PATH = "creditcard.csv"   # gerekirse değiştir
df = pd.read_csv(CSV_PATH)
print("Şekil:", df.shape)
print(df.head(3))

assert "Class" in df.columns, "Class etiketi bulunamadı"

drop_cols = []
for cand in ["CustomerID", "customer_id", "ID", "id"]:
    if cand in df.columns:
        drop_cols.append(cand)

y = df["Class"].values.astype(int)
X = df.drop(columns=["Class"] + drop_cols)

print("\nKullanılan özellik sayısı:", X.shape[1])
```

```python
# ===========================================
# 2) Ölçekleme (StandardScaler)
# ===========================================
# NEDEN? SOM öklid mesafesi kullanır; ölçek farkları sonucu bozar.
#       Tüm sayısal sütunları standardize ediyoruz (mean=0, std=1).
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train/Test böl (etiket sadece değerlendirme için)
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, stratify=y, random_state=42
)

print("Split:", X_train.shape, X_test.shape,
      " | train fraud rate:", y_train.mean(), " | test fraud rate:", y_test.mean())
```

```python
# ===========================================
# 3) SOM Eğitimi
# ===========================================
# Grid boyutunu veri büyüklüğüne göre "sıkı" tutuyoruz (12x12 / 15x15 iyi başlar).
# Sigma: Komşuluk genişliği (1.5-2.0 arası genelde iyi)
# Learning rate: 0.5 ile başla, MiniSom iterasyonla düşürür.
m, n = 12, 12
sigma = 1.8
lr = 0.5
iterations = 15 * X_train.shape[0]  # 10-20x N_train iyi pratik; hızlı test için 5x seçilebilir

som = MiniSom(x=m, y=n, input_len=X_train.shape[1], sigma=sigma, learning_rate=lr,
              neighborhood_function='gaussian', random_seed=42)
som.random_weights_init(X_train)
print("SOM init tamam.")

som.train_random(X_train, iterations, verbose=True)
print("SOM eğitim bitti.")
```

```python
# ===========================================
# 4) U-Matrix (Distance Map) Görselleştirme
# ===========================================
# NEDEN? Hücreler arası mesafe ısı haritası; yüksek mesafeli bölgeler sınır/anomali adacıkları olabilir.
u_matrix = som.distance_map()  # (m, n)

plt.figure(figsize=(6,6))
plt.imshow(u_matrix.T, cmap="bone", origin="lower")
plt.title("SOM U-Matrix (Distance Map)")
plt.colorbar()
plt.show()
```

```python
# ===========================================
# 5) Skor-1: Quantization Error (QE) tabanlı anomaly score
# ===========================================
# NEDEN? Her örneğin en iyi eşleşen hücre (BMU) ağırlığına uzaklığı = "ne kadar uymuyor" = anomali skoru
def som_quantization_error(som, x):
    w = som.get_weights()       # (m, n, d)
    bmu = som.winner(x)         # (i, j)
    w_bmu = w[bmu]              # (d,)
    return np.linalg.norm(x - w_bmu)

scores_qe = np.array([som_quantization_error(som, x) for x in X_test])

roc_qe = roc_auc_score(y_test, scores_qe)
pr_qe  = average_precision_score(y_test, scores_qe)
print(f"[QE] ROC-AUC: {roc_qe:.4f} | PR-AUC: {pr_qe:.4f}")
```

```python
# ===========================================
# 6) Skor-2: BMU "fraud yoğunluğu" (semi-supervised propensity)
# ===========================================
# NEDEN? Unsupervised SOM üzerinde, sadece skor fonksiyonunu etiketle kalibre ediyoruz:
#       Her BMU hücresi için train set fraud oranını hesapla; testte o hücreye düşenlere bu oranı skor olarak ver.
from collections import defaultdict

bmu_train = np.array([som.winner(x) for x in X_train])

cell_counts = defaultdict(int)
cell_fraud  = defaultdict(int)
for (cx, cy), lab in zip(bmu_train, y_train):
    cell_counts[(cx, cy)] += 1
    cell_fraud[(cx, cy)]  += int(lab)

cell_rate = {k: (cell_fraud[k] / cell_counts[k]) for k in cell_counts.keys()}

bmu_test = np.array([som.winner(x) for x in X_test])
scores_prop = np.array([cell_rate.get(tuple(c), 0.0) for c in bmu_test])

roc_prop = roc_auc_score(y_test, scores_prop)
pr_prop  = average_precision_score(y_test, scores_prop)
print(f"[BMU-Propensity] ROC-AUC: {roc_prop:.4f} | PR-AUC: {pr_prop:.4f}")
```

```python
# ===========================================
# 7) Skorları birleştirme (opsiyonel basit ortalama)
# ===========================================
# NEDEN? QE ve Propensity farklı sinyaller taşır; basit bir ensemble çoğu zaman daha iyi sonuç verir.
scores_combo = 0.5 * ( (scores_qe - scores_qe.min()) / (scores_qe.ptp()+1e-12) ) + \
               0.5 * ( (scores_prop - scores_prop.min()) / (scores_prop.ptp()+1e-12) )

roc_combo = roc_auc_score(y_test, scores_combo)
pr_combo  = average_precision_score(y_test, scores_combo)
print(f"[COMBO] ROC-AUC: {roc_combo:.4f} | PR-AUC: {pr_combo:.4f}")
```

```python
# ===========================================
# 8) Eşik seçimi: PR eğrisinden (F1-opt veya Precision>=x)
# ===========================================
# NEDEN? "En anormal %1" gibi sabit top-k yerine, PR eğrisinden F1'i maksimize eden eşik genelde daha dengeli sonuç verir.

def pick_threshold_by_f1(y_true, scores):
    prec, rec, thr = precision_recall_curve(y_true, scores)
    f1s = 2*prec[:-1]*rec[:-1] / (prec[:-1] + rec[:-1] + 1e-12)
    i = np.argmax(f1s)
    return thr[i], prec[i], rec[i], f1s[i]

def pick_threshold_by_precision(y_true, scores, target_p=0.80):
    prec, rec, thr = precision_recall_curve(y_true, scores)
    idx = np.where(prec[:-1] >= target_p)[0]
    if len(idx) == 0:
        # hedef precision sağlanmıyorsa F1-opt'a düş
        return pick_threshold_by_f1(y_true, scores)
    i = idx[np.argmax(rec[idx])]
    return thr[i], prec[i], rec[i], 2*prec[i]*rec[i]/(prec[i]+rec[i]+1e-12)

# Hangi skorla karar vereceğiz? (QE / PROP / COMBO)
chosen = scores_combo  # çoğu zaman COMBO en iyi olur; istersen değiştir: scores_qe veya scores_prop

t_f1, p_f1, r_f1, f1_f1 = pick_threshold_by_f1(y_test, chosen)
pred_f1 = (chosen >= t_f1).astype(int)

print("\n[F1-opt threshold]")
print("threshold:", round(float(t_f1), 6), "| P:", round(float(p_f1),3),
      "| R:", round(float(r_f1),3), "| F1:", round(float(f1_f1),3))
print(classification_report(y_test, pred_f1, digits=4))

# Örnek: Precision ≥ 0.80 şartıyla eşik
t_p, p_p, r_p, f1_p = pick_threshold_by_precision(y_test, chosen, target_p=0.80)
pred_p = (chosen >= t_p).astype(int)

print("\n[Precision>=0.80 threshold]")
print("threshold:", round(float(t_p), 6), "| P:", round(float(p_p),3),
      "| R:", round(float(r_p),3), "| F1:", round(float(f1_p),3))
print(classification_report(y_test, pred_p, digits=4))
```

```python
# ===========================================
# 9) ROC ve PR eğrileri (seçtiğin skor için)
# ===========================================
from sklearn.metrics import auc

def plot_curves(y_true, scores, title_suffix="(Combo)"):
    prec, rec, _ = precision_recall_curve(y_true, scores)
    fpr, tpr, _ = roc_curve(y_true, scores)
    ap  = average_precision_score(y_true, scores)
    roc = roc_auc_score(y_true, scores)

    plt.figure(figsize=(11,4))
    plt.subplot(1,2,1)
    plt.plot(fpr, tpr); plt.plot([0,1],[0,1],'k--')
    plt.title(f"ROC {title_suffix} | AUC={roc:.3f}")
    plt.xlabel("FPR"); plt.ylabel("TPR"); plt.grid(alpha=0.3)

    plt.subplot(1,2,2)
    plt.plot(rec, prec)
    plt.title(f"PR {title_suffix} | AP={ap:.3f}")
    plt.xlabel("Recall"); plt.ylabel("Precision"); plt.grid(alpha=0.3)
    plt.tight_layout(); plt.show()

plot_curves(y_test, scores_combo, "(COMBO)")
```

```python
# ===========================================
# 10) K-Means karşılaştırma: Elbow + küme fraud oranları
# ===========================================
# NEDEN? Gözetimsiz bir referans yöntemle SOM sonuçlarını kıyaslamak için.
inertias, ks = [], range(2, 13)
for k in ks:
    km = KMeans(n_clusters=k, n_init=10, random_state=42)
    km.fit(X_train)
    inertias.append(km.inertia_)

plt.plot(ks, inertias, marker="o")
plt.title("K-Means Elbow (Inertia)")
plt.xlabel("k"); plt.ylabel("inertia"); plt.grid(alpha=0.3)
plt.show()

k = 8  # istersen elbow grafiğine göre değiştir
km = KMeans(n_clusters=k, n_init=10, random_state=42).fit(X_train)
train_k = km.predict(X_train)
test_k  = km.predict(X_test)

k_rates = {i: (y_train[train_k==i].mean() if (train_k==i).any() else 0.0) for i in range(k)}
k_score = np.array([k_rates[c] for c in test_k])

print("[KMeans-Propensity] ROC-AUC:", roc_auc_score(y_test, k_score))
print("[KMeans-Propensity] PR-AUC :", average_precision_score(y_test, k_score))

pd.DataFrame({"cluster": list(k_rates.keys()),
              "fraud_rate": [k_rates[i] for i in k_rates]}).sort_values("fraud_rate", ascending=False)
```

---

## Kılavuz (özet)

* **Kimlik kolonlarını at** (örn. `CustomerID`)
* **Sadece StandardScaler** ile başla (PCA’yı sonra dene)
* **SOM’i kompakt kur** (12×12 / 15×15; `sigma≈1.8`, `iter≈10–20×N`)
* **İki skor üret:**

  * **QE (BMU uzaklığı)** → saf unsupervised anomali sinyali
  * **BMU fraud yoğunluğu** → semi-supervised “propensity” sinyali
* **Skorları birleştir**, **etikete bakarak eşiği PR eğrisinden seç** (F1-opt veya Precision≥x)
* **K-Means** ile kıyasla; istersen **ensemble** yap

Kodı çalıştır; çıkan **ROC/PR**, **rapor** ve **fraud-rate kümeleri**’ni paylaş — birlikte hiperparametreleri (harita boyutu/sigma/iterasyon, eşik seçimi, skor birleştirme ağırlıkları) ince ayarlarız.
